# Tidy Data in Python
The examples and code in this notebook are made by [Jean-Nicholas Hould](http://www.jeannicholashould.com/)

Detailed explanations for important code snippets are provided by Mervat Abuelkheir as part of the CSEN1095 Data Engineering Course.

The goal of this notebook is to show how a messy dataset can be tidied into proper rows representing objects, columns representing attributes, and cells representing scalar values.

Pay attention to the <span style="color:red"> <b> paragraphs in bold red</b></span>; they ask you to do something and provide input!

First thing we need to do is import some libraries.

In [1]:
import pandas as pd
import datetime # to handle date/time attributes
from os import listdir # os is a module for interacting with the OS
from os.path import isfile, join # to verify file object, and concatenate paths
import glob # to find pathnames matching a specific pattern
import re # regular expressions :)

## Examining the datasets

In this part of the exercise we will import a number of datasets and examine their structure to verify if the datasets are tidy.

Remember the requirements for a tidy dataset:
<br> 1- Each row describes a single object
<br> 2- Each column describes a property/attribute of that object
<br> 3- Column values have the same measurement unit
<br> 4- Columns contain atomic/scalar values (no multiple values per table cell)

For each dataset imported, test your ability to identify is it is tidy or not.

### Dataset 1: Pew Research Center

Pew Research Center is a famous center in the US that performs polling surveys on citizens. This is example data about the breakdown of yearly income per religion.

In [2]:
df = pd.read_csv("./data/pew-raw.csv")
df

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58
3,Catholic,418,617,732,670,638,1116
4,Dont know/refused,15,14,15,11,10,35
5,Evangelical Prot,575,869,1064,982,881,1486
6,Hindu,1,9,7,9,11,34
7,Historically Black Prot,228,244,236,238,197,223
8,Jehovahs Witness,20,27,24,24,21,30
9,Jewish,19,19,25,25,30,95


<span style="color:red"> <b> What are the attributes of interest? How are they organized? Is the dataset tidy? </b></span> 
    
You can brainstorm your thought process and document in a new cell if you like.
<br>Instructions for beginners:
<br>- Add a new cell from the notebook menu above (+ button).
<br>- Double click anywhere inside the new cell to enter edit mode.
<br>- When done, press CTRL+ENTER or SHIFT+ENTER to commit content.
<br>- You can edit content anytime by double clicking inside the cell.

## Let's tidy the dataset!

The melt function is used to change the format of a pandas data frame from wide to long, assigning one column as an identifier and "unpivoting" the others.

In [3]:
# melt method takes as input a dataframe, one or more identifier attributes, one or more attribute names, and value attribute 
# define new pandas dataframe, religion column will be identifier attribute
# values spread across multiple column headers of income ranges will be unpivoted into new attribute "income"
# actual frequencies of citizens with specific income range will be unpivoted into new attribute "freq"
formatted_df = pd.melt(df,["religion"], var_name="income", value_name="freq")
formatted_df = formatted_df.sort_values(by=["religion"]) # just sorting the new table by religion attribute
formatted_df.head(10) # show first 10 rows

,religion,income,freq
0,Agnostic,<$10k,27
30,Agnostic,$30-40k,81
40,Agnostic,$40-50k,76
50,Agnostic,$50-75k,137
10,Agnostic,$10-20k,34
20,Agnostic,$20-30k,60
41,Atheist,$40-50k,35
21,Atheist,$20-30k,37
11,Atheist,$10-20k,27
31,Atheist,$30-40k,52


<span style="color:red"> <b> Why do the indices that are added automatically by pandas appear out of order? </b></span> 
<br>(Just a question to let you think of how pandas dataframes are indexed.)

### Dataset 2: Billboard Top 100

This dataset outlines data about the top hit songs on the Billboard list. 

In [4]:
df = pd.read_csv("./data/billboard.csv", encoding="mac_latin2")
df.head(10)

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2000,Janet,Doesn't Really Matter,4:17,Rock,2000-06-17,2000-08-26,59,52.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2000,Destiny's Child,Say My Name,4:31,Rock,1999-12-25,2000-03-18,83,83.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2000,"Iglesias, Enrique",Be With You,3:36,Latin,2000-04-01,2000-06-24,63,45.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2000,Sisqo,Incomplete,3:52,Rock,2000-06-24,2000-08-12,77,66.0,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,81,54.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<span style="color:red"> <b> Again: What are the attributes of interest? How are they organized? Is the dataset tidy? </b></span>

The structure of the dataset is more complex than the previous one, and it is not immediately clear what a typical row should represent or look like. Answering the above questions helps you frame the data better. 
<br>You can brainstorm your thought process and document in a new cell if you like.

## Let's tidy the dataset!

One way a record could be organized is to make it represent the rank of each song in every week the song was on the Billboard list. This omits the need to keep track of all 76 weeks data, which is null for most of the songs.

A record would have data about the year, artist, track, time, genre, week, rank, and date.

The unique identifier is no single attribute, as one artist can have the track on the billboards at the same year, genre, and time. The only difference would be the week, rank, and date (since date is correlated with week). Therefore, to identify a track's rank and week, we need to use the year, artist, track, time, genre, and date as a combined unique identifier.


### <span style="color:blue"> Note on conversions in Python</span>

<span style="color:blue"> The following conversions are accepted by Python:</span>
<br><span style="color:blue"> - passing a string representation of an integer into int</span>
<br><span style="color:blue"> - passing a string representation of a float into float</span>
<br><span style="color:blue"> - passing a string representation of an integer into float</span>
<br><span style="color:blue"> - passing an integer into float</span>
<br><span style="color:blue"> - passing a float into int</span>

<span style="color:blue"> You get an error if you pass a string representation of a float (or anything other than an integer) into int</span>
<br><span style="color:blue"> This is especially problematic if you have NaN values that are float and you want to convert them to integers. It does not work using int, and you have to use Int32. </span>

Now back to tidying up the Billboard dataset!

In [5]:
# Melting
# Define unique identifiers in one variable. Include both dates of entry and peak for now; will be merged into one attribute later.
id_vars = ["year","artist.inverted","track","time","genre","date.entered","date.peaked"]
# Now melt structure to have identifiers, variable name (week) and values (rank)
df = pd.melt(frame=df,id_vars=id_vars, var_name="week", value_name="rank")

# Formatting 
# First, for week attribute, extract week number from string representation of week column names and convert to float then to integer
df["week"] = df["week"].str.extract('(\d+)', expand=False).astype(float).astype(int) 
# Second, extract rank values and convert them to integer
df["rank"] = df["rank"].astype('Int32')

# Cleaning out unnecessary rows
df = df.dropna()

# Create "date" columns
# Date for each week is date the track entered the billboard + number of weeks passed for an entry
# Example: if date entered is 26/02/2000, then this is the date for week 1, and the date will change for week 2 to become 04/03/2000, and so on
df["date"] = pd.to_datetime(df["date.entered"]) + pd.to_timedelta(df["week"], unit='w') - pd.DateOffset(weeks=1)


# Frame the final tidy data, replacing the dates of entry and peak with only the date, then sort by the identifiers
final_df = df[["year", "artist.inverted", "track", "time", "genre", "week", "rank", "date"]]
final_df = final_df.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])

# Assigning the tidy dataset to a variable for future usage
billboard = final_df

<span style="color:red"><b>Why did we convert the week string to float before converting it to int?</b></span>

<span style="color:red"><b>What does the parameter '(\d+)' in the string.extract method do? </b></span>

In [6]:
# Now let's check the tidied data frame
# Separating this line of code to avoid running the formatting code multiple times and getting errors
final_df.head(10)

,year,artist.inverted,track,time,genre,week,rank,date
246,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,1,87,2000-02-26
563,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,2,82,2000-03-04
880,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,3,72,2000-03-11
1197,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,4,77,2000-03-18
1514,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,5,87,2000-03-25
1831,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,6,94,2000-04-01
2148,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,7,99,2000-04-08
287,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,1,91,2000-09-02
604,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,2,87,2000-09-09
921,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,3,92,2000-09-16


### Dataset 3: Tubercolosis

This dataset outlines the number of tubercolosis patients in different countries in the year 2000.

A few notes on the raw data set:

- The columns starting with "m" or "f" contain multiple variables: 
    - Sex ("m" or "f")
    - Age Group ("0-14","15-24", "25-34", "45-54", "55-64", "65", "unknown")
- Mixture of 0s and missing values("NaN"). This is due to the data collection process and the distinction is important for this dataset.

In [7]:
df = pd.read_csv("./data/tb-raw.csv")
df

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,NaN,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,NaN,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,NaN,3.0
5,AM,2000,2.0,152.0,130.0,131.0,63,26,21.0,NaN,1.0
6,AN,2000,0.0,0.0,1.0,2.0,0,0,0.0,NaN,0.0
7,AO,2000,186.0,999.0,1003.0,912.0,482,312,194.0,NaN,247.0
8,AR,2000,97.0,278.0,594.0,402.0,419,368,330.0,NaN,121.0
9,AS,2000,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN


<span style="color:red"> <b> Again: What are the attributes of interest? How are they organized? Is the dataset tidy? </b></span>

## Let's tidy the dataset!

Same as what we did before: We need identifiers, we need the column names to represent variables (two in this case, since the column names carry information about gender and age group), and we need the frequency values to be in one column.


In [8]:
# Let's use the year and country as unique identifiers, and name the # of patients as "cases" and the column variables as "sex and age"
df = pd.melt(df, id_vars=["country","year"], value_name="cases", var_name="sex_and_age")

# Extract Sex, Age lower bound and Age upper bound group
tmp_df = df["sex_and_age"].str.extract("(\D)(\d+)(\d{2})", expand=False)    

# tmp_df now has multiple columns corresponding to the strings extracted from the column names. Now name the columns
tmp_df.columns = ["sex", "age_lower", "age_upper"]

# Create "age" column based on "age_lower" and "age_upper"
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]

# Merge - axis parameter indicates the axis along which merge will take place. 1 means by columns
df = pd.concat([df, tmp_df], axis=1)

# Drop unnecessary columns and rows
df = df.drop(['sex_and_age',"age_lower","age_upper"], axis=1)
# Drop null values
df = df.dropna()
# Sort rows by all four attributes
df = df.sort_values(ascending=True,by=["country", "year", "sex", "age"])
df.head(10)

,country,year,cases,sex,age
0,AD,2000,0.0,m,0-14
10,AD,2000,0.0,m,15-24
20,AD,2000,1.0,m,25-34
30,AD,2000,0.0,m,35-44
40,AD,2000,0.0,m,45-54
50,AD,2000,0.0,m,55-64
81,AE,2000,3.0,f,0-14
1,AE,2000,2.0,m,0-14
11,AE,2000,4.0,m,15-24
21,AE,2000,4.0,m,25-34


<span style="color:red"><b>What does the parameter value "(\D)(\d+)(\d{2})" do?</b></span>

### Dataset 4: Global Historical Climatology Network

In [9]:
df = pd.read_csv("./data/weather-raw.csv")
df.head(10)

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN
5,MX17004,2010,3,tmin,NaN,NaN,NaN,NaN,14.2,NaN,NaN,NaN
6,MX17004,2010,4,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MX17004,2010,4,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MX17004,2010,5,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,MX17004,2010,5,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In this dataset, variables are stored in both rows and columns. tmax and tmin stand for max and min temperatures for each day. Date is broken down to three columns, with the day being spread across multiple columns. We need the data to represent min and max temperatures per date.

Notice that the dataset has many missing values.

## Let's tidy the dataset!

Same as what we did before: We need identifiers, we need the column names to represent variables (min and max, and date!), and we need the temperature values to be in two columns.


In [10]:
# Let's start first by putting the day values in one column. We will not play with min and max temperatures for now
df = pd.melt(df, id_vars=["id", "year","month","element"], var_name="day_raw")
df.head(10)

,id,year,month,element,day_raw,value
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN
5,MX17004,2010,3,tmin,d1,NaN
6,MX17004,2010,4,tmax,d1,NaN
7,MX17004,2010,4,tmin,d1,NaN
8,MX17004,2010,5,tmax,d1,NaN
9,MX17004,2010,5,tmin,d1,NaN


In [11]:
# Extracting day
# df["day"] automatically adds a "day" attribute to the df dataframe
df["day"] = df["day_raw"].str.extract("d(\d+)", expand=False)  
df["id"] = "MX17004"

# Convert year, month, and day to numeric values
# Notice the use of the lamda function to apply one instruction to multiple inputs
df[["year","month","day"]] = df[["year","month","day"]].apply(lambda x: pd.to_numeric(x, errors='ignore'))

# Let's define a function to create a date from the different columns. Function accepts a row of 3 values as input and returns consolidated date
def create_date_from_year_month_day(row):
    return datetime.datetime(year=row["year"], month=int(row["month"]), day=row["day"])

# Define date attribute, by having the temporary lamda function call the create_date function
df["date"] = df.apply(lambda row: create_date_from_year_month_day(row), axis=1)
# Drop the redundant columns used to compute date
df = df.drop(['year',"month","day", "day_raw"], axis=1)
# Now drop the missing values
df = df.dropna()

# Unmelting column "element"
df = df.pivot_table(index=["id","date"], columns="element", values="value")
df.reset_index(drop=False, inplace=True)
df

element,id,date,tmax,tmin
0,MX17004,2010-02-02,27.3,14.4
1,MX17004,2010-02-03,24.1,14.4
2,MX17004,2010-03-05,32.1,14.2


## <span style="color:red"> Exercise your tidying muscles! </span>

<span style="color:red"><b> The GapMinder dataset includes information about the life expectancy, the GDP per capita, and the population of various countries between the years 1952 and 2007.</b></span>

<span style="color:red"> <b>Import the dataset, investigate it to identify what the potential attributes should be, the problems with the current structure, and think of how to tidy the dataset, and then proceed to tidy the dataset.</b></span>

In [14]:
import pandas as pd
import datetime # to handle date/time attributes
from os import listdir # os is a module for interacting with the OS
from os.path import isfile, join # to verify file object, and concatenate paths
import glob # to find pathnames matching a specific pattern
import re # regular expressions :)

df = pd.read_csv("./data/gapminder.csv")
df

,continent,country,gdpPercap_1952,gdpPercap_1957,gdpPercap_1962,gdpPercap_1967,gdpPercap_1972,gdpPercap_1977,gdpPercap_1982,gdpPercap_1987,...,pop_1962,pop_1967,pop_1972,pop_1977,pop_1982,pop_1987,pop_1992,pop_1997,pop_2002,pop_2007
0,Africa,Algeria,2449.008185,3013.976023,2550.816880,3246.991771,4182.663766,4910.416756,5745.160213,5681.358539,...,11000948.0,12760499.0,14760787.0,17152804.0,20033753.0,23254956.0,26298373.0,29072015.0,31287142,33333216
1,Africa,Angola,3520.610273,3827.940465,4269.276742,5522.776375,5473.288005,3008.647355,2756.953672,2430.208311,...,4826015.0,5247469.0,5894858.0,6162675.0,7016384.0,7874230.0,8735988.0,9875024.0,10866106,12420476
2,Africa,Benin,1062.752200,959.601080,949.499064,1035.831411,1085.796879,1029.161251,1277.897616,1225.856010,...,2151895.0,2427334.0,2761407.0,3168267.0,3641603.0,4243788.0,4981671.0,6066080.0,7026113,8078314
3,Africa,Botswana,851.241141,918.232535,983.653976,1214.709294,2263.611114,3214.857818,4551.142150,6205.883850,...,512764.0,553541.0,619351.0,781472.0,970347.0,1151184.0,1342614.0,1536536.0,1630347,1639131
4,Africa,Burkina Faso,543.255241,617.183465,722.512021,794.826560,854.735976,743.387037,807.198586,912.063142,...,4919632.0,5127935.0,5433886.0,5889574.0,6634596.0,7586551.0,8878303.0,10352843.0,12251209,14326203
5,Africa,Burundi,339.296459,379.564628,355.203227,412.977514,464.099504,556.103265,559.603231,621.818819,...,2961915.0,3330989.0,3529983.0,3834415.0,4580410.0,5126023.0,5809236.0,6121610.0,7021078,8390505
6,Africa,Cameroon,1172.667655,1313.048099,1399.607441,1508.453148,1684.146528,1783.432873,2367.983282,2602.664206,...,5793633.0,6335506.0,7021028.0,7959865.0,9250831.0,10780667.0,12467171.0,14195809.0,15929988,17696293
7,Africa,Central African Republic,1071.310713,1190.844328,1193.068753,1136.056615,1070.013275,1109.374338,956.752991,844.876350,...,1523478.0,1733638.0,1927260.0,2167533.0,2476971.0,2840009.0,3265124.0,3696513.0,4048013,4369038
8,Africa,Chad,1178.665927,1308.495577,1389.817618,1196.810565,1104.103987,1133.984950,797.908101,952.386129,...,3150417.0,3495967.0,3899068.0,4388260.0,4875118.0,5498955.0,6429417.0,7562011.0,8835739,10238807
9,Africa,Comoros,1102.990936,1211.148548,1406.648278,1876.029643,1937.577675,1172.603047,1267.100083,1315.980812,...,191689.0,217378.0,250027.0,304739.0,348643.0,395114.0,454429.0,527982.0,614382,710960


In [22]:
df_gdp = pd.read_csv("./data/gapminder/gdp.csv")
df_gdp = pd.melt(df_gdp,["continent","country"], var_name="Year", value_name="GDP Value")
df_gdp["Year"] = df_gdp["Year"].str.extract('(\d+)', expand=False).astype(float).astype(int) 
df_gdp

,continent,country,Year,GDP Value
0,Africa,Algeria,1952,2449.008185
1,Africa,Angola,1952,3520.610273
2,Africa,Benin,1952,1062.752200
3,Africa,Botswana,1952,851.241141
4,Africa,Burkina Faso,1952,543.255241
5,Africa,Burundi,1952,339.296459
6,Africa,Cameroon,1952,1172.667655
7,Africa,Central African Republic,1952,1071.310713
8,Africa,Chad,1952,1178.665927
9,Africa,Comoros,1952,1102.990936


In [16]:
df_le = pd.read_csv("./data/gapminder/le.csv")
df_le = pd.melt(df_le,["continent","country"], var_name="Year", value_name="Life Expectancy")
df_le["Year"] = df_le["Year"].str.extract('(\d+)', expand=False).astype(float).astype(int) 
df_le

,continent,country,Year,Life Expectancy
0,Africa,Algeria,1952,43.077
1,Africa,Angola,1952,30.015
2,Africa,Benin,1952,38.223
3,Africa,Botswana,1952,47.622
4,Africa,Burkina Faso,1952,31.975
5,Africa,Burundi,1952,39.031
6,Africa,Cameroon,1952,38.523
7,Africa,Central African Republic,1952,35.463
8,Africa,Chad,1952,38.092
9,Africa,Comoros,1952,40.715


In [17]:
df_pop = pd.read_csv("./data/gapminder/pop.csv")
df_pop = pd.melt(df_pop,["continent","country"], var_name="Year", value_name="Population")
df_pop["Year"] = df_pop["Year"].str.extract('(\d+)', expand=False).astype(float).astype(int) 
df_pop

,continent,country,Year,Population
0,Africa,Algeria,1952,9279525.0
1,Africa,Angola,1952,4232095.0
2,Africa,Benin,1952,1738315.0
3,Africa,Botswana,1952,442308.0
4,Africa,Burkina Faso,1952,4469979.0
5,Africa,Burundi,1952,2445618.0
6,Africa,Cameroon,1952,5009067.0
7,Africa,Central African Republic,1952,1291695.0
8,Africa,Chad,1952,2682462.0
9,Africa,Comoros,1952,153936.0


In [23]:
df_tidied = pd.concat([df_gdp, df_le[["Life Expectancy"]], df_pop[["Population"]]], axis=1)
df_tidied

,continent,country,Year,GDP Value,Life Expectancy,Population
0,Africa,Algeria,1952,2449.008185,43.077,9279525.0
1,Africa,Angola,1952,3520.610273,30.015,4232095.0
2,Africa,Benin,1952,1062.752200,38.223,1738315.0
3,Africa,Botswana,1952,851.241141,47.622,442308.0
4,Africa,Burkina Faso,1952,543.255241,31.975,4469979.0
5,Africa,Burundi,1952,339.296459,39.031,2445618.0
6,Africa,Cameroon,1952,1172.667655,38.523,5009067.0
7,Africa,Central African Republic,1952,1071.310713,35.463,1291695.0
8,Africa,Chad,1952,1178.665927,38.092,2682462.0
9,Africa,Comoros,1952,1102.990936,40.715,153936.0


In [13]:
dfnew = df_tidied
dfnew.to_csv(r"data/gapminder/tidiedgapminder.csv", index = False)